# Загрузка и обработка данных

In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
#!curl -o dataset.tskv https://raw.githubusercontent.com/yandex/geo-reviews-dataset-2023/master/geo-reviews-dataset-2023.tskv

# !git clone https://github.com/yandex/geo-reviews-dataset-2023.git

In [ ]:
def parse_tskv_line(line):
    return dict(item.split('=', 1) for item in line.strip().split('\t'))

with open('geo-reviews-dataset-2023/geo-reviews-dataset-2023.tskv', 'r', encoding='utf-8') as file:
    data = [parse_tskv_line(line) for line in file]

df = pd.DataFrame(data)

df.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [ ]:
rating_balance = df['rating'].value_counts()
rating_balance

rating
5.    390515
4.     41160
1.     34351
3.     21686
2.     12088
0.       200
Name: count, dtype: int64

In [ ]:
df = df[df['rating'] != '0.']

df["rating"] = df["rating"].apply(lambda x: int(x[0]) - 1)

df.rename(columns={'rating': 'labels'}, inplace=True)

df = df.fillna('')

In [ ]:
df

,address,name_ru,labels,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,2,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,4,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,0,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,3,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,4,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...
...,...,...,...,...,...
499995,"Москва, Южный административный округ, район Би...",Бирюлёво-Пассажирская,3,Железнодорожная станция,"Охрана кривая но добрая, двери не закрываются ..."
499996,"Москва, Южный административный округ, район Би...",Бирюлёво-Пассажирская,3,Железнодорожная станция,По сравнению со многими современными платформа...
499997,"Новосибирск, Коммунистическая улица, 48А",,4,"Бар, паб","Приятная атмосфера, прекрасное вино, волшебная..."
499998,"Астраханская область, Харабалинский район",Сарай-Бату,4,Достопримечательность,Был с семьёй 13.06.23 Отличное место. Рекоменд...


In [ ]:
df['text'] = df['text']+'[SEP]'+df['address']+'[SEP]'+df['name_ru']+'[SEP]'+df['rubrics']
df = df.drop(columns = ['address', 'name_ru', 'rubrics'])

# Загрузка тестовых данных и разметка

In [ ]:
with open('test.tskv', 'r', encoding='utf-8') as file:
    data = [parse_tskv_line(line) for line in file]

df_test = pd.DataFrame(data)
df_test.head()

,address,name_ru,rubrics,text
0,"Московская обл., Электроугли, ул. Школьная, вб...",Школьная,Жилой комплекс,Как то он выгледит по другому и не так красиво...
1,"Москва, Ленинский проспект, 34/1",Банк ВТБ,Банк,Не решили вопрос \nВ приложении втб прислали с...
2,"Москва, улица Плеханова, 17, стр. 6",Перово Плаза,Гостиница,"Ужасное место,при заезде не могли никак засели..."
3,"Москва, Каширское шоссе, 57, корп. 4",Пятёрочка,Супермаркет,Всегда нереально медленные кассиры
4,"Москва, Головинское шоссе, 5, корп. 1",О'кей,Продуктовый гипермаркет;Гипермаркет,Самое ужасное мясо на развес. Всегда продают т...


## Подготовка датасета к обучению

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("seninoseno/rubert-base-cased-sentiment-study-feedbacks-solyanka")
max_length = 512

dataset = Dataset.from_pandas(df[['text', 'labels']])
dataset = dataset.shuffle(seed=42)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/499800 [00:00<?, ? examples/s]

In [ ]:
train_test_split0 = tokenized_datasets.train_test_split(test_size=0.2)
train_test_split = train_test_split0['train'].train_test_split(test_size=0.1)

train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("seninoseno/rubert-base-cased-sentiment-study-feedbacks-solyanka", ignore_mismatched_sizes=True ,num_labels=5)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, AutoModelForSequenceClassification

from torch import nn

class CustomTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(self.model.device)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")

        if labels is None:
            raise ValueError("Labels are missing or incorrectly extracted from the inputs.")

        labels = labels.to(model.device)

        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fn = nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss

labels = torch.tensor(train_dataset['labels'])

class_counts = torch.bincount(labels)
class_weights = 1.0 / class_counts.float()
class_weights = class_weights / class_weights.sum()

In [ ]:
learning_rate = 1e-5
gradient_accumulation_steps = 13
per_device_train_batch_size = 16
num_train_epochs = 3

best_training_args = TrainingArguments(
    num_train_epochs=3,
    output_dir="test_trainer",
    learning_rate=1e-5,
    gradient_accumulation_steps=13,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    report_to='tensorboard',
    load_best_model_at_end=True,
    metric_for_best_model='eval_accuracy',
    greater_is_better=True,
)

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=1)

In [ ]:
trainer = CustomTrainer(
    class_weights=class_weights,
    model=model,
    args=best_training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,0.838200,0.644292,0.843838


KeyboardInterrupt: 

In [ ]:
eval_dataset = eval_dataset.remove_columns(["text"])
eval_dataset.set_format("torch")
eval_dataset = eval_dataset.remove_columns("__index_level_0__")

In [ ]:
from torch.utils.data import DataLoader

eval_dataloader = DataLoader(eval_dataset, shuffle=True, batch_size=32)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(len(eval_dataloader)))
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
metric = evaluate.load("accuracy")
model.eval()
cnt = 0
for batch in eval_dataloader:
    cnt+=1
    if cnt == 500:
      break
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar.update(1)
metric.compute()

  0%|          | 0/1250 [00:00<?, ?it/s]

{'accuracy': 0.811184869739479}

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch


def predict(text, address, name_ru, rubrics):
    input_text = f"{text}"

    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()

    return predicted_class

In [ ]:
def predict_rating(row):
    predicted_class = predict(row['text'], row['address'], row['name_ru'], row['rubrics'])
    return str(predicted_class + 1) + '.'

df_test['rating'] = df_test.apply(predict_rating, axis=1)

with open('test_with_rating.tskv', 'w', encoding='utf-8') as file:
    for index, row in df_test.iterrows():
        line = ''
        for column in df_test.columns:
            line += f'{column}={row[column]}\t'
        print(line.strip(), end='\n', file=file)

with open('test_with_rating.tskv', 'r', encoding='utf-8') as file:
    data = [parse_tskv_line(line) for line in file]

df_test = pd.DataFrame(data)
df_test.head()

,address,name_ru,rubrics,text,rating
0,"Московская обл., Электроугли, ул. Школьная, вб...",Школьная,Жилой комплекс,Как то он выгледит по другому и не так красиво...,2.
1,"Москва, Ленинский проспект, 34/1",Банк ВТБ,Банк,Не решили вопрос \nВ приложении втб прислали с...,1.
2,"Москва, улица Плеханова, 17, стр. 6",Перово Плаза,Гостиница,"Ужасное место,при заезде не могли никак засели...",1.
3,"Москва, Каширское шоссе, 57, корп. 4",Пятёрочка,Супермаркет,Всегда нереально медленные кассиры,2.
4,"Москва, Головинское шоссе, 5, корп. 1",О'кей,Продуктовый гипермаркет;Гипермаркет,Самое ужасное мясо на развес. Всегда продают т...,1.


In [ ]:
trainer.save_model("model_gpu12_02")